In [5]:
import requests
import pandas as pd
from time import sleep
import numpy as np
from tqdm import tqdm
import os

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [6]:
import pickle
from lyricsgenius import Genius

In [7]:
import re
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Genius config

In [8]:
load_dotenv()

True

In [9]:
token = os.getenv("genius-token")
genius = Genius(token)

Check if genius is configured succesfully:

In [10]:
artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
print(artist.songs)

Searching for songs by Andy Shauf...

Song 1: "Alexander All Alone"
"All Of My Love" is not valid. Skipping.
Song 2: "All the Same"
Song 3: "Angela"

Reached user-specified song limit (3).
Done. Found 3 songs.
[Song(id, artist, ...), Song(id, artist, ...), Song(id, artist, ...)]


# Importing pickle file

In [11]:
with open('../data/pickle/dataset_viral150_unique.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [12]:
tracks.head()

,url,artist_clean,track_clean
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [13]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.head()

,url,artist,track
8246,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
8247,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
8248,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
8249,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
8250,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


# Selenium

Configuro mi pestaña de selenium

In [14]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://chromedriver.chromium.org/downloads')

[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:02<00:00, 2.91MB/s]


Voy a probar que me funcione la búsqueda de un elemento

In [15]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
## si quieres tardar más, le pones un sleep
## sleep(20)
## acepto los cookies
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(1)
## busco el nombre del actor que quiero buscar
driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys('drake fake love', Keys.ENTER)
sleep(1)
driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
driver.implicitly_wait(30)
## extraigo la información de la letra de cada canción
lyrics = driver.find_element('css selector', '#lyrics-root > div:nth-child(3)').text
lyrics2 = driver.find_element('css selector', '#lyrics-root > div:nth-child(6)').text
sleep(1)
release_date = driver.find_element('css selector', '#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.fqavsD > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div:nth-child(2) > div').text
#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.kEkpYM > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div:nth-child(3) > div

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:02<00:00, 2.89MB/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.fqavsD > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div:nth-child(2) > div"}
  (Session info: chrome=109.0.5414.74)
Stacktrace:
Backtrace:
	(No symbol) [0x00736643]
	(No symbol) [0x006CBE21]
	(No symbol) [0x005CDA9D]
	(No symbol) [0x00601342]
	(No symbol) [0x0060147B]
	(No symbol) [0x00638DC2]
	(No symbol) [0x0061FDC4]
	(No symbol) [0x00636B09]
	(No symbol) [0x0061FB76]
	(No symbol) [0x005F49C1]
	(No symbol) [0x005F5E5D]
	GetHandleVerifier [0x009AA142+2497106]
	GetHandleVerifier [0x009D85D3+2686691]
	GetHandleVerifier [0x009DBB9C+2700460]
	GetHandleVerifier [0x007E3B10+635936]
	(No symbol) [0x006D4A1F]
	(No symbol) [0x006DA418]
	(No symbol) [0x006DA505]
	(No symbol) [0x0

ESTOY TRABAJANDO AQUÍ
Clicando el read more

In [20]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
## si quieres tardar más, le pones un sleep
## sleep(20)
## acepto los cookies
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(1)
## busco el nombre del actor que quiero buscar
driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys('drake fake love', Keys.ENTER)
sleep(1)
driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
driver.implicitly_wait(30)
## clico el all credits
driver.find_element('css selector', '#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.fqavsD > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div.HeaderMetadata__Section-sc-1p42fnf-2.glevmK > div > a').click()
sleep(1)

CONTINUO AQUÍ CON B4S

In [26]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
## si quieres tardar más, le pones un sleep
## sleep(20)
## acepto los cookies
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(1)
## busco el nombre del actor que quiero buscar
driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys('drake fake love', Keys.ENTER)
sleep(1)
driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
driver.implicitly_wait(30)
## clico el all credits
driver.find_element('css selector', '#application > main > div.SongPageGriddesktop-sc-1px5b71-0.jecoie.SongHeaderdesktop__Container-sc-1effuo1-0.fqavsD > div.SongHeaderdesktop__Column-sc-1effuo1-1.SongHeaderdesktop__Right-sc-1effuo1-4.ePANpp > div > div.HeaderMetadata__Grid-sc-1p42fnf-1.ePueMf > div.HeaderMetadata__Section-sc-1p42fnf-2.glevmK > div > a').click()
sleep(1)
page_source = driver.page_source
url = driver.current_url
res = requests.get(url + '#song-info')
soup = BeautifulSoup(res.content, 'html.parser')
pagina = soup.text


In [27]:
pagina

'\n\n\nDrake – Fake Love Lyrics | Genius Lyrics\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFeaturedChartsVideosPromote Your MusicSign UpFake LoveDrakeTrack 20 on\xa0More Life\xa0“Fake Love” finds Drake criticizing friends and associates who only show him love because of his incredible success. He identifies their false emotions and explains how he… Read More\xa0Produced byVinylz & GingRelease DateOctober 23, 2016View All Credits\xa033.8M217193TranslationsEspañolFake Love Lyrics[Chorus]I\'ve been down so long, it look like up to meThey look up to meI got fake people showin\' fake love to meStraight up to my face, straight up to my faceI\'ve been down so long, it look like up to meThey look up to meI got fake people showin\' fake love to meStraight up to my face, straight up to my face[Verse 1]Somethin\' ain\'t right when we talkin\'Somethin\' ain\'t right when we talkin\'Look like you hidin\' your problemsRe

In [ ]:
pagina

In [14]:
release_date

'October 23, 2016'

In [ ]:
lyrics + lyrics2

In [ ]:
canciones = {
    'lyrics': [],
    'release_date': []
}

In [ ]:
for index, row in tracks.iterrows():
    print(row['artist'].split(',')[0],'---', row['track'])
    ## iniciamos el web driver
    opciones= Options()
    opciones.add_argument("--start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://genius.com/')
    driver.maximize_window()
    driver.set_window_size(1920, 1080)
    ## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
    ##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
    driver.implicitly_wait(30)
    ## si quieres tardar más, le pones un sleep
    ## sleep(20)
    ## acepto los cookies
    driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
    sleep(1)
    ## busco el nombre del actor que quiero buscar
    driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'], Keys.ENTER)
    sleep(1)
    driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a').click()
    driver.implicitly_wait(30)
    ## extraigo la información de la letra de cada canción
    all_lyrics = ''
    for item in range(1,15):
        try:
            lyrics = driver.find_element('css selector', f'#lyrics-root > div:nth-child({item})').text
            if len(lyrics) > 12:
                all_lyrics += lyrics
            else:
                continue
        except:
            continue
    canciones['lyrics'].append(all_lyrics)
    sleep(1)
    try:
        release_date = driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[3]/div[1]/div[2]/div/div[1]/span[1]/span').text
    except:
        release_date = driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[3]/div').text
    canciones['release_date'].append(release_date)

In [ ]:
canciones

# Beautiful Soup
Prueba

In [18]:
canciones = {
    'lyrics': [],
    'release_date': []
}

In [49]:
url = 'https://genius.com/Drake-one-dance-lyrics#song-info'

In [50]:
res = requests.get(url)
res ## tiene que dar 200 

<Response [200]>

In [51]:
type(res.content) ## para consultarel html de la página

bytes

In [52]:
sopa = BeautifulSoup(res.content, 'html.parser')
type(sopa) ## es más amigable, pero sigue siendo html

bs4.BeautifulSoup

Datos canciones

In [23]:
pagina = sopa.text
re.findall(r'Date(\w+ \d+, \d+)', pagina)[0]

'April 5, 2016'

In [53]:
pagina

'\n\n\nDrake – One Dance Lyrics | Genius Lyrics\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFeaturedChartsVideosPromote Your MusicSign UpOne DanceDrakeTrack 12 on\xa0Views\xa0“One Dance” is an R&B song with dancehall and Afrobeats inflections, featuring a slowed down sample of the vocals and chord stabs from a 2008 UK Funky house anthem, Kyla’s… Read More\xa0FeaturingKyla & WizkidProduced byDJ Maphorisa, 40 & 2 moreRelease DateApril 5, 2016View All Credits\xa016.1M290233TranslationsTürkçeEspañolEnglishDeutschFrançaisOne Dance Lyrics[Refrain: Kyla]Baby, I like your style[Verse 1: Drake]Grips on your waist, front way, back wayYou know that I don\'t playStreets not safe but I never run awayEven when I\'m awayOti, otiThere\'s never much love when we go OTI pray to make it back in one pieceI pray, I pray[Chorus: Drake]That\'s why I need a one danceGot a Hennessy in my handOne more time \'fore I goHigher powers ta

# Beautiful soup + selenium

In [ ]:
canciones = {
    'lyrics': [],
    'release_date': []
}

In [ ]:
for index, row in tracks.iterrows():
    print(row['artist'].split(',')[0],'---', row['track'])
    ## iniciamos el web driver
    opciones= Options()
    opciones.add_argument("--start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://genius.com/')
    driver.maximize_window()
    driver.set_window_size(1920, 1080)
    ## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
    ##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
    driver.implicitly_wait(30)
    ## si quieres tardar más, le pones un sleep
    ## sleep(20)
    ## acepto los cookies
    driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
    sleep(1)
    ## busco el nombre de canción que quiero buscar
    driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'], Keys.ENTER)
    sleep(1)
    ## selecciono el resultado de la búsqueda que quiero
    driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a').click()
    driver.implicitly_wait(30)
    ## extraigo la información de la letra de cada canción
    page_source = driver.page_source
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    pagina = sopa.text
    canciones['release_date'].append(re.findall(r'Date(\w+ \d+, \d+)', pagina)[0])
    #break

In [ ]:
canciones

In [ ]:
tracks

In [ ]:
urls = []
for index, row in tracks.iterrows():
    urls.append(f"https://genius.com/{row['artist'].split(',')[0].replace(' ','-')}-{row['track'].replace(' ','-')}")
urls[10:15]

# test

In [47]:
genius_search_url = f"http://api.genius.com/search?q={'drake one love'}&access_token={token}"

In [44]:
response = requests.get(genius_search_url)
json_data = response.json()

In [45]:
json_data

{'meta': {'status': 200},
 'response': {'hits': [{'highlights': [],
    'index': 'song',
    'type': 'song',
    'result': {'annotation_count': 1,
     'api_path': '/songs/4329719',
     'artist_names': 'Union Jack',
     'full_title': "Don't Test Me by\xa0Union\xa0Jack",
     'header_image_thumbnail_url': 'https://images.genius.com/41006413d9d2672bdea99e9f05b2ca06.300x310x1.png',
     'header_image_url': 'https://images.genius.com/41006413d9d2672bdea99e9f05b2ca06.496x512x1.png',
     'id': 4329719,
     'language': 'en',
     'lyrics_owner_id': 7970562,
     'lyrics_state': 'complete',
     'path': '/Union-jack-dont-test-me-lyrics',
     'pyongs_count': None,
     'relationships_index_url': 'https://genius.com/Union-jack-dont-test-me-sample',
     'release_date_components': {'year': 2018, 'month': 11, 'day': 1},
     'release_date_for_display': 'November 1, 2018',
     'release_date_with_abbreviated_month_for_display': 'Nov. 1, 2018',
     'song_art_image_thumbnail_url': 'https://imag

# Export

In [ ]:
with open('../data/pickle/genius_dataset.pickle', 'wb') as data_spotify:
    pickle.dump(tracks, data_spotify)